In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

from utils.load_data import load_gt_data
from statsmodels.tsa.seasonal import seasonal_decompose

%load_ext autoreload
%autoreload 2
%reload_ext autoreload

In [ ]:
SEED = 42

In [ ]:
gt_data = load_gt_data()

gt_data['date'] = pd.to_datetime(gt_data['date'])
gt_data = gt_data.sort_values(by='date')

In [ ]:
gt_cols = [col for col in gt_data.columns if col not in ['date', 'country']]

gt_data_melted = gt_data.melt(id_vars=['date', 'country'], value_vars=gt_cols)

gt_data_melted

In [ ]:
def plot_overall_trend(data, country, title=None, col=None, col_label=None, new_figs=True, plot_average=True):
    
    cols = [col for col in data.columns if col not in ['date', 'country']]

    data_melted = data.melt(id_vars=['date', 'country'], value_vars=cols)

    country_mask = data_melted['country'] == country
    zero_mask = data_melted['value'] != 0

    if col is not None:
        if new_figs:
            plt.figure(figsize=(8,4))

        mask = country_mask & zero_mask & (data_melted['variable'] == cols[-1])

        plt.title(f'Google trends in {country} for {col}')

        sns.lineplot(data_melted[mask], x='date', y='value', linewidth=1, label=col_label)
    else:
        if new_figs:
            plt.figure(figsize=(14,4))

        masks = {}
        for i in range(6):
            masks[i] = country_mask & zero_mask & data_melted['variable'].isin(cols[(i*15):((i+1)*15)])

        if title is not None:
            plt.title(title)
        else:
            plt.title(f'Google trends in {country}')

        for i in range(6):
            plt.subplot(int(f"32{i+1}"))
            sns.lineplot(data_melted[masks[i]], x='date', y='value', hue='variable', legend=None, linewidth=1, alpha=0.75)
            if plot_average:
                plt.plot(data_melted[masks[i]]['date'].unique(), data_melted[masks[i]].groupby('date')['value'].agg("mean"), color='b')

        plt.show()

        if plot_average:
            plt.figure(figsize=(8, 2))
            plt.ylim(0, 100)
            plt.title(f"Overall trend in all countries")
            plt.plot(data_melted[zero_mask]['date'].unique(), data_melted[zero_mask].groupby('date')['value'].agg("mean"), color='b')
            plt.show()

plot_overall_trend(gt_data, "United States")


## 1st approach : seasonal decompose average over columns

In [ ]:
def get_average_trend(data, plot=False):
    data = data.copy()

    gt_cols = [col for col in data.columns if col not in ['date', 'country']]

    data = data.drop('country', axis=1).groupby('date').mean()
    
    trends, seasonals, residuals = {}, {}, {}
    for col in gt_cols:
        decompose_result = seasonal_decompose(data[col], model="additive", period=4)

        trends[col] = decompose_result.trend
        seasonals[col] = decompose_result.seasonal
        residuals[col] = decompose_result.resid

    mean_trend = np.mean([trends[col] for col in trends], axis=0)
    if plot:
        plt.figure(figsize=(8, 4))

        for col in trends:
            sns.lineplot(x=data.index, y=trends[col], color='gray', alpha=0.3, linewidth=1)

        sns.lineplot(x=data.index, y=mean_trend, color='red', label="Average trend")
        plt.title('Average trend over all Google Trends (using seasonal_decompose)')

    return mean_trend



mean_trend = get_average_trend(gt_data, plot=True)

## 2nd approach (OECD paper) : HP filter then PCA

In [ ]:
from statsmodels.tsa.filters.hp_filter import hpfilter
from sklearn.decomposition import PCA

def detrend_gts(data, plot=False):
    data = data.copy()
    
    gt_cols = [col for col in data.columns if col.endswith('_average')]

    if plot:
        plt.figure(figsize=(15, 4))
    
    # Compute the log of the mean SVI values across countries
    means_over_countries = data.drop('country', axis=1).groupby('date').mean()
    log_svi = means_over_countries.apply(np.log1p)

    mean_over_time = log_svi.mean(axis=1)
    log_svi_average_mean, log_svi_average_std = mean_over_time.mean(), mean_over_time.std()

    if plot:
        plt.subplot(131)
        plt.plot(mean_over_time)
        plt.title('Mean of the mean log-SVI values\nacross countries and categories')

    # Apply HP filtering to extract long-term trend
    hp_output = log_svi.apply(lambda x: hpfilter(x, lamb=1600)[1])  # Long-term trend
    if plot:
        plt.subplot(132)
        plt.plot(hp_output)
        plt.title('High-pass filter output')

    # Normalize the long-term trends for PCA
    mean, std = hp_output.mean(axis=1).values, hp_output.std(axis=1).values
    hp_output = (hp_output - mean[:, None]) / std[:, None]

    # Apply PCA to extract the common component
    pca = PCA(n_components=1)
    common_component = pca.fit_transform(hp_output)

    # Rescale PCA component to the same scale as log-SVI
    common_component = (common_component - np.mean(common_component)) / np.std(common_component)
    common_component = common_component * log_svi_average_std + log_svi_average_mean  # Rescaled to match log-SVI

    if plot:
        plt.subplot(133)
        plt.plot(common_component)
        plt.title('Common component extracted by PCA,\nrescaled to match log-SVI')
        plt.show()

    # Subtract the common component from each series
    zero_mask = data[gt_cols] != 0
    data[gt_cols] = np.log1p(data[gt_cols])
    unique_dates = data['date'].unique()
    common_component_map = pd.Series(common_component.ravel(), index=unique_dates)
    
    data['common_component'] = data['date'].map(common_component_map)
    data[gt_cols] = data[gt_cols].sub(data['common_component'], axis=0)
    data.drop(columns=['common_component'], inplace=True)

    # Return to original SVI scale
    data[gt_cols] = np.expm1(data[gt_cols])
    gt_min, gt_max = np.min(data[gt_cols].values), np.max(data[gt_cols].values)
    data[gt_cols] = (data[gt_cols]) / (gt_max - gt_min) * 100
    data[gt_cols] = data[gt_cols].where(zero_mask, 0) # Keep zeros (no data)

    return data

detrended_gt_data = detrend_gts(gt_data, plot=True)

# Results

In [ ]:
plot_overall_trend(gt_data, "United States", title="Original data for the United States", plot_average=False)
plot_overall_trend(detrended_gt_data, "United States", title="Detrended data for the United States", plot_average=False)